In [61]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, fixed
from skimage import data, filters, morphology
from functools import reduce


def plot(size, images):
    """Plots the given images in a grid of given size."""
    row, col = size
    plt.figure(figsize=(12, 8))
    plt.suptitle("Canny Edge Detector")
    for i, (name, cmap, image) in enumerate(images, 1):
        plt.subplot(row, col, i, title=name)
        plt.imshow(image, cmap=cmap)
        plt.colorbar()
        plt.axis("off")


def is_maximum(orientations, magnitudes, direction):
    dy, dx = direction

    grad_in_dir = np.cos(np.arctan2(dy, dx) - orientations) > np.cos(np.deg2rad(22.5))
    grad_opp_dir = np.cos(np.arctan2(-dy, -dx) - orientations) > np.cos(
        np.deg2rad(22.5)
    )
    is_max_backwards = magnitudes > np.roll(magnitudes, shift=(dy, dx), axis=(0, 1))
    is_max_forwards = magnitudes > np.roll(magnitudes, shift=(-dy, -dx), axis=(0, 1))

    return (grad_in_dir | grad_opp_dir) & is_max_backwards & is_max_forwards


@interact(
    sigma=widgets.FloatSlider(min=0.1, max=6.0, step=0.1, value=3.0),
    tlo=widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.3),
    thi=widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.75),
    hist_steps=widgets.IntSlider(min=1, max=200, value=100),
    image=fixed(data.cat().astype(np.float32) / 255),
)
def canny(image, sigma, tlo, thi, hist_steps):
    """Canny edge detection"""
    # The neighborhood used for the hystersis step, to connect weak pixels to strong ones
    hysteresis_neibourhood = np.full((3, 3), True)
    # The directions in which to apply non-maximum suppression
    supression_neighborhood = [(1, 0), (0, 1), (1, 1), (1, -1)]

    # convert image to greyscale
    image_gray = np.mean(image, axis=2)
    # Blur the image to get rid of noise that could be detected as edge
    image_blurred = filters.gaussian(image_gray, sigma)

    # Apply edge detection filter (sobel)
    edges_h = filters.sobel_h(image_blurred)
    edges_v = filters.sobel_v(image_blurred)

    # Extract edge orientation and magnitude
    edges_orient = np.arctan2(edges_h, edges_v)
    edges_magnitude = np.sqrt(edges_h**2 + edges_v**2) / 2

    # Get binary mask of edge values that are a maximum in the direction of their
    # orientation
    maximums_along_grad_direction = reduce(
        lambda acc, nei: acc | is_maximum(edges_orient, edges_magnitude, nei),
        supression_neighborhood,
        False,
    )

    # Select only the edge magnitudes that are the maximum
    max_magnitudes = edges_magnitude * maximums_along_grad_direction
    # remap the intensity range to (0,1)
    max_magnitudes_stretched = (max_magnitudes - np.min(max_magnitudes)) / np.ptp(
        max_magnitudes
    )
    # apply threshold to select strong pixels
    strong_pixels = max_magnitudes_stretched > thi
    # and weak pixels
    weak_pixels = (thi > max_magnitudes_stretched) & (max_magnitudes_stretched > tlo)

    # Hysteresis: Repeatedly mark weak pixels as strong if they are in the neighborhood of
    # a strong pixel.
    hysteresis = strong_pixels.copy()
    steps = 0
    final_result = True
    for _ in range(hist_steps):
        strong_neibours = morphology.binary_dilation(hysteresis, hysteresis_neibourhood)
        weak_neibours = strong_neibours & weak_pixels

        # stop if nothing changes anymore
        if np.all(hysteresis[weak_neibours]):
            break

        hysteresis[weak_neibours] = True
        steps += 1
    else:
        final_result = False

    plot(
        (4, 3),
        [
            ("Original Image", "grey", image),
            ("Greyscale Image", "grey", image_gray),
            ("Blurred Image", "grey", image_blurred),
            ("Horizontal Edges", "grey", edges_h),
            ("Vertical Edges", "grey", edges_v),
            ("Total Edges", "grey", edges_magnitude),
            ("Edge Orientation", "rainbow", edges_orient),
            ("Is Maximum?", "grey", maximums_along_grad_direction),
            ("Maximum Magnitudes", "grey", max_magnitudes),
            ("Strong Magnitudes", "grey", strong_pixels),
            ("Weak Magnitudes", "grey", weak_pixels),
            (
                (
                    f"Histeresis (complete after {steps} steps)"
                    if final_result
                    else f"Histeresis (first {steps} steps)"
                ),
                "grey",
                hysteresis,
            ),
        ],
    )

interactive(children=(FloatSlider(value=3.0, description='sigma', max=6.0, min=0.1), FloatSlider(value=0.3, de…